In [1]:
from skimage.color import rgb2hsv
from skimage.util import img_as_float64
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.metrics import structural_similarity as ssim

In [2]:
def slightest(img1,img2):
    if len(img1.shape)==2 and len(img2.shape)==2:
        pass
    else:
        print("bad image shape, input must be gray image")
        return None
    return np.minimum(img1,img2)

In [3]:
def getMinChannel(img):
    if len(img.shape)==3 and img.shape[2]==3:
        pass
    else:
        print("bad image shape, input must be color image")
        return None
    
    return np.min(img, axis=2)

In [4]:
def printImage(_list,Name_list):
    axes=[]
    fig=plt.figure(figsize=(16,14))
    for a in range(len(_list)):
        b = _list[a]
        axes.append( fig.add_subplot(2,2, a+1) )
        subplot_title=("Subplot"+str(Name_list[a]))
        axes[-1].set_title(subplot_title)  
        plt.imshow(b,cmap="gray")
    fig.tight_layout()
    plt.show()

In [5]:
class Node(object):
    def __init__(self,x,y,value):
        self.x = x
        self.y = y
        self.value = value

    def printInfo(self):
        print('%s:%s:%s' %(self.x,self.y,self.value))

def getAtomsphericLight(darkChannel,img,meanMode = False, percent = 0.001):
    
    size = darkChannel.shape[0]*darkChannel.shape[1]
    height = darkChannel.shape[0]
    width = darkChannel.shape[1]

    nodes = []

    #
    for i in range(height):
        for j in range(width):
            oneNode = Node(i,j,darkChannel[i,j])
            nodes.append(oneNode)	

    #
    nodes = sorted(nodes, key = lambda node: node.value,reverse = True)
    # print(nodes[0].value)
    # print(nodes[1].value)
    # print(nodes[2].value)
    # print(nodes[3].value)
    # print(nodes[4].value)
    # print(nodes[len(nodes)-1].value)

    atomsphericLight = 0

    # #
    if int(percent*size) == 0:
        for i in range(3):
            if img[nodes[0].x,nodes[0].y,i] > atomsphericLight:
                atomsphericLight = img[nodes[0].x,nodes[0].y,i]
        return atomsphericLight

    # #
    # if meanMode:
    #     sum = 0
    #     for i in range(0,int(percent*size)):
    #         for j in range(0,3):
    #             sum = sum + img[nodes[i].x,nodes[i].y,j]
    #     atomsphericLight = int(sum/(int(percent*size)*3))
    #     return atomsphericLight

    #0.1%(percent)
    for i in range(int(percent*size)):
        for j in range(3):
            if img[nodes[i].x,nodes[i].y,j] > atomsphericLight:
                atomsphericLight = img[nodes[i].x,nodes[i].y,j]
    return atomsphericLight

In [6]:
def Dehazing(img,imgReal):
    down_width = 705#735
    down_height = 480#504
    down_points = (down_width, down_height)
    img_resized = cv2.resize(img, down_points, interpolation= cv2.INTER_LINEAR)

        
    ## HSV openCV
    resultCV = cv2.cvtColor(img_resized, cv2.COLOR_BGR2HSV)
    channel_H,channel_S,channel_V = cv2.split (resultCV)

    down_points = (down_width, down_height)
    img_resizedReal = cv2.resize(imgReal, down_points, interpolation= cv2.INTER_LINEAR)

    resultCVReal = cv2.cvtColor(img_resizedReal, cv2.COLOR_BGR2HSV)
    channelReal_H,channelReal_S,channelReal_V = cv2.split (resultCVReal)

    minChannel = getMinChannel(img_resized)
    kernelR1 = np.ones((31,31), np.uint8)  
    img_erosion = cv2.erode(minChannel, kernelR1, iterations=1)  

    kernelR2 = np.ones((35,35), np.uint8)  
    img_dilation = cv2.dilate(img_erosion, kernelR2, iterations=1)

    img_open = img_dilation.copy()
    Imodify = slightest(minChannel,img_open)

    A_ = getAtomsphericLight(Imodify,img_resized, percent = 0.1)

    imgDark = np.float64(Imodify)
    omega = 0.9 #ω is used to preserve distant haze, usually fixed as 0.9.
    transmission = 1 - (omega * (imgDark / A_))
    transmission_for_Saturation = transmission.copy()

    t0 = 0.1
    transmission[transmission<t0] = t0

    channel_VFloat = np.float64(channel_V)
    numeratorR = channel_VFloat - A_
    denominatorR = transmission
    recoValue = A_ + (numeratorR/denominatorR)
    recoValue = np.uint8(recoValue)

    transmission_for_Saturation = transmission_for_Saturation - 1
    maximumRGB = channel_V.copy()
    equation_1 = A_ * transmission_for_Saturation
    equation_2 = equation_1/maximumRGB
    channel_SFloat = np.float64(channel_S)
    recoSaturation = channel_SFloat * (1/(1+equation_2))
    recoSaturation = np.uint8(recoSaturation)


    mergedDehazing = cv2.merge([channel_H,recoSaturation,recoValue])
    bgrimg = cv2.cvtColor(mergedDehazing, cv2.COLOR_HSV2BGR)

    return img_resizedReal,img_resized,bgrimg


In [13]:
#img = cv2.imread('D:/EntornosPython/imgs/Middlebury_Hazy/Piano_Hazy.bmp')
#img = cv2.imread('D:/EntornosPython/imgs/Fog/6726971629.png')
#img = cv2.imread('D:/EntornosPython/imgs/Data_Landscape/hazy/55_hazy.png')
img = cv2.imread('D:/EntornosPython/imgs/HAZY_NTIRE_2018/hazy/17_indoor_hazy.jpg')
#img = cv2.imread('D:/EntornosPython/imgs/Middlebury_Hazy/Flowers_Hazy.bmp')

#imgReal = cv2.imread('D:/EntornosPython/imgs/Middlebury_GT/Piano_im0.png')
#imgReal = cv2.imread('D:/EntornosPython/imgs/Fog/827916289.png')
imgReal = cv2.imread('D:/EntornosPython/imgs/HAZY_NTIRE_2018/GT/17_indoor_GT.jpg')
#imgReal = cv2.imread('D:/EntornosPython/imgs/Middlebury_GT/Flowers_im0.png')

img_resizedReal,img_resized,bgrimg = Dehazing(img,imgReal)

In [14]:
cv2.imshow("Image Ground Truth", img_resizedReal)
cv2.imshow("Hazy Image", img_resized)
cv2.imshow("Dezahing Image ", bgrimg)
cv2.waitKey (0)
cv2.destroyAllWindows()